I first saved my Data Query from Stack Exchange to my google storage bucket from the terminal of the master node.

Now I will load my data into spark.

In [1]:
stack_df = spark.read.csv("gs://spark-1-dataproc/data", header=True, inferSchema=True)

Now I will look at my DataFrame

In [2]:
stack_df.show(10)

+--------+--------------------+-----+----------+
|      Id|                Tags|Score|ClosedDate|
+--------+--------------------+-----+----------+
|69969780|  <dataframe><julia>|    5|      null|
|69969782|<jquery><ajax><as...|    1|      null|
|69969783|<javascript><time...|    0|      null|
|69969784|                null|    1|      null|
|69969785|<python><python-3...|    0|      null|
|69969786|<c#><visual-studi...|    0|      null|
|69969787|                null|    0|      null|
|69969788|   <node.js><server>|    0|      null|
|69969789|<microsoft-graph-...|    0|      null|
|69969791|                null|    1|      null|
+--------+--------------------+-----+----------+
only showing top 10 rows



In [3]:
print("There are {} rows in the stack_df DataFrame.".format(stack_df.count()))

print("There are {} columns in the stack_df DataFrame and their names are {}.".format(len(stack_df.columns), stack_df.columns))


There are 50000 rows in the stack_df DataFrame.
There are 4 columns in the stack_df DataFrame and their names are ['Id', 'Tags', 'Score', 'ClosedDate'].


In [502]:
# I am interested in posts with no tag so I will fill all Null Values with "None"
stack_df_clean = stack_df.na.fill(value='None', subset=["Tags"])

# Now I really don't think these characters <> 
# are going to help me much. Let's get rid of those.
stack_df_clean = stack_df_clean.withColumn('Tags', regexp_replace('Tags', '><', ' '))
stack_df_clean = stack_df_clean.withColumn('Tags', regexp_replace('Tags', '<', ''))
stack_df_clean = stack_df_clean.withColumn('Tags', regexp_replace('Tags', '>', ''))

stack_df_clean.show(2)

+--------+--------------------+-----+----------+
|      Id|                Tags|Score|ClosedDate|
+--------+--------------------+-----+----------+
|69969780|     dataframe julia|    5|      null|
|69969782|jquery ajax asp.n...|    1|      null|
+--------+--------------------+-----+----------+
only showing top 2 rows



In [554]:
# First, I only want to map the Tags. 
# So I will create a sub-dataframe of Tags.
stack_df_tags = stack_df_clean.select('Tags') 

# Let's see if we can make this DataFrame into an RDD.
from pyspark.rdd import RDD
stack_df_rdd = stack_df_tags.rdd

# and map the Keys(Tags) and values(count).
from pyspark.sql.functions import split
tags = stack_df_rdd.flatMap(lambda row: [(tag, 1) for tag in row[0].split()]). \
                reduceByKey(lambda total, count: total + count)

# Let's see this as a DataFrame so we can do some ploting
tagColumns = ["Tag", "Count"]
df_tags = tags.toDF(tagColumns).toPandas().to_csv('gs://spark-1-dataproc/data/tmp/spark_output/tags.csv')

In [552]:
# Now, I only want to map the Ids to the Tags. 
# So I will create a sub-dataframe of Ids and Tags.
stack_df_id = stack_df_clean.select('Id', "Tags")

# Let's see if we can make this DataFrame into an RDD.
stack_df_id_rdd = stack_df_id.rdd

# and map the Keys(Tags, PostID) and values(count).
tags_plus_id = stack_df_id_rdd.flatMap(lambda row: [((row[0], tag), 1) for tag in row[1].split()]). \
                    reduceByKey(lambda total, count: total + count)

# Let's see this as a DataFrame so we can do some ploting
idColumns = ["PostId, Tag", "Count"]
df_id = tags_plus_id.toDF(idColumns).toPandas().to_csv('gs://spark-1-dataproc/data/tmp/spark_output/id.csv')

In [553]:
# Lastly, I only want to map the Scores to the Tags. 
# So I will create a sub-dataframe of Scores and Tags.
stack_tags_score = stack_df_clean.select('Tags', 'Score')

# Let's see if we can make this DataFrame into an RDD.
tag_score_rdd = stack_tags_score.rdd

# and map the Keys(Tags, Scores) and values(count).
tags_plus_scores = tag_score_rdd.flatMap(lambda row: [((tag, row[1]), 1) for tag in row[0].split()]). \
                    reduceByKey(lambda total, count: total + count)

# Let's see this as a DataFrame so we can do some ploting
scoreColumns = ["Tag, Score", "Count"]
df_scores = tags_plus_scores.toDF(scoreColumns).toPandas().to_csv('gs://spark-1-dataproc/data/tmp/spark_output/scores.csv')